In [85]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')

True

In [86]:
from openai import OpenAI

client = OpenAI()

image_url = "https://ihe-delft-ihe-website-production.s3.eu-central-1.amazonaws.com/s3fs-public/styles/530x530/public/2022-11/Sea%20level%20rise%20Vietnam.jpg"

image_url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYe-ZtojD1lIN1uOwCP3afMx0YRfccIWbLIw&s'

In [87]:
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

# ---------------------------------------------------------
# 1. Define a tool that returns an image URL
# ---------------------------------------------------------
tools = [
    {
        "type": "function",
        "name": "get_random_image",
        "description": "Return a random image URL",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "decription of image",
                },
            },
            "required": ["topic"],
        },
    },
]

def get_random_image(topic):
    # Hardcoded image URL
    return image_url


# ---------------------------------------------------------
# 2. Initial user input
# ---------------------------------------------------------
input_list = [
    {
        "role": "user",
        "content": "Get the poster of a random things from internet"
    }
]

response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Add model output (tool call) to conversation
input_list += response.output

print('input_list after initial response: ', input_list)
# ---------------------------------------------------------
# 3. Execute tool & return image URL as tool output
# ---------------------------------------------------------
for item in response.output:
    if item.type == "function_call" and item.name == "get_random_image":
        args = json.loads(item.arguments)
        image_url = get_random_image(args["topic"])

        input_list.append({
            "type": "function_call_output",
            "call_id": item.call_id,
            "output": [{
                "type": "input_image",
                "image_url": image_url,
            }]
        })

print("Final input:")
print(input_list)

input_list after initial response:  [{'role': 'user', 'content': 'Get the poster of a random things from internet'}, ResponseReasoningItem(id='rs_083c72d1c12a4ad100693f075076fc81979294bcedf6f8f910', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"topic":"poster featuring random everyday objects"}', call_id='call_tsJJHjs8Q6QYuxVeR1mbIfnz', name='get_random_image', type='function_call', id='fc_083c72d1c12a4ad100693f0757057081979e9af2c4122d766d', status='completed')]
Final input:
[{'role': 'user', 'content': 'Get the poster of a random things from internet'}, ResponseReasoningItem(id='rs_083c72d1c12a4ad100693f075076fc81979294bcedf6f8f910', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"topic":"poster featuring random everyday objects"}', call_id='call_tsJJHjs8Q6QYuxVeR1mbIfnz', name='get_random_image', type='function_call', id='fc_083c72d1c12a4ad

In [88]:

# ---------------------------------------------------------
# 4. Final LLM call using tool output
# ---------------------------------------------------------
response = client.responses.create(
    model="gpt-5",
    instructions="Explain the image retuned by the tool. Do not go by the name of the file. Clearly specify only what you understand by reading the contents of the image",
    tools=tools,
    input=input_list,
)

print("\nFinal output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)


Final output:
{
  "id": "resp_083c72d1c12a4ad100693f0758fba0819785e4f39affb58abc",
  "created_at": 1765738329.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Explain the image retuned by the tool. Do not go by the name of the file. Clearly specify only what you understand by reading the contents of the image",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_083c72d1c12a4ad100693f075a4b00819793e22c432e1274a4",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_083c72d1c12a4ad100693f076c13188197a8f91f9645fb0645",
      "content": [
        {
          "annotations": [],
          "text": "Here’s what I see in the image: four people outdoors, all wearing white shirts, black pants, and red gloves, working together to pick up trash and place it into a large black garbage bag along a path surrounded by trees

In [97]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI, ChatGoogleGenerativeAI
from langchain_core.tools import tool
from openai.types import image
from langchain.messages import ToolMessage


@tool
def get_random_image(topic: str) -> str:
    '''return random image url based on the topic'''
    return image_url



# llm = ChatOpenAI(
#     model='gpt-4o-mini', use_responses_api=True
# )

# llm = ChatGroq(
#     model='meta-llama/llama-4-scout-17b-16e-instruct',
#     # model_kwargs={'use_responses_api': True}
# )
import os

llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-pro', vertexai=False
    )

llm = llm.bind_tools([get_random_image])
# llm = llm.bind_tools([{"google_maps": {}}])


In [98]:
input_list = [
    {
        "role": "user",
        "content": "Get the poster of a random things from internet and explain what you see in the image in detail and tell me no of peoples you see in the image. if you are not able to see the image clearly tell so"
    }
]

In [99]:
res = llm.invoke(input_list)
res

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_random_image', 'arguments': '{"topic": "random things"}'}, '__gemini_function_call_thought_signatures__': {'315758b7-4a16-4276-b0bb-3b5e474c747f': 'CvgKAXLI2ny7ritsvkh+nF2qfNTFuf7VyzvZrwMgiUBSHvhXBOYr3Xvb5sG9I4d4T1bsz0ZN8YHlxH2q6YXxKktQF72yszMCmwvPmOMAcESyQU/IyDHp3fZ8T9hlOPPhsgBXa4N6XtBCDrwlLMEOXqMPWmjRTM6PPjMCTnK/Ts2qncv5Py78s7XtNtKQ7KkImFbX9ADQ7tFzi4xgoCxlUdXK3OBD+ge4zh8nqX+6ZSYhmC9exOlHtPKKILJM2r+z2qI86+4K+UCoYlubR6iazZQj36KO/wEuKjnuGuTe3zI1ySXjBKW68MB0nYwCEFHQytALHKWEh+9TWcy9jB6SWZVuFTPh0T5DRKt/ibo6MImY8EIh9DotAj8yDTdFgehNWfFMs7sD2PFl8jGEQpGIyWBzKpf3H+PpEVMKHYXM/CNYvH6lnCOZFLcdwCstjRVuTgvEjtzyNmesklNbgt97pxNuvVwAYseuqI84QCs9YbEBCtJKCJhnCKNeNfU/sWq6bvWKuYXiDnVoby8emSrGKIO1hPz+jXnc5JfCWDkSeUvQ/ZfHyFNknwvmMYppxdiScKVDOzifdHzOZZI+DJj8xizoQYps1FKEMl1hxMz9AvEfvKhhpOucq698Xl2R+0O1i2p3HkB6FXeMsnnEW1ueCPigb2JTQepHSbaDJlO/dtxnrQ2YYj80EaW7b8bMVzcq9uVgPp1gAMmTI5GZaogxkSV5+Qvuw+antJ8DBXfWr/evgbO3bkEboZO7oWJLj4KMfIDqi2o1nuzs

In [100]:
input_list.append(res)
input_list

[{'role': 'user',
  'content': 'Get the poster of a random things from internet and explain what you see in the image in detail and tell me no of peoples you see in the image. if you are not able to see the image clearly tell so'},
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_random_image', 'arguments': '{"topic": "random things"}'}, '__gemini_function_call_thought_signatures__': {'315758b7-4a16-4276-b0bb-3b5e474c747f': 'CvgKAXLI2ny7ritsvkh+nF2qfNTFuf7VyzvZrwMgiUBSHvhXBOYr3Xvb5sG9I4d4T1bsz0ZN8YHlxH2q6YXxKktQF72yszMCmwvPmOMAcESyQU/IyDHp3fZ8T9hlOPPhsgBXa4N6XtBCDrwlLMEOXqMPWmjRTM6PPjMCTnK/Ts2qncv5Py78s7XtNtKQ7KkImFbX9ADQ7tFzi4xgoCxlUdXK3OBD+ge4zh8nqX+6ZSYhmC9exOlHtPKKILJM2r+z2qI86+4K+UCoYlubR6iazZQj36KO/wEuKjnuGuTe3zI1ySXjBKW68MB0nYwCEFHQytALHKWEh+9TWcy9jB6SWZVuFTPh0T5DRKt/ibo6MImY8EIh9DotAj8yDTdFgehNWfFMs7sD2PFl8jGEQpGIyWBzKpf3H+PpEVMKHYXM/CNYvH6lnCOZFLcdwCstjRVuTgvEjtzyNmesklNbgt97pxNuvVwAYseuqI84QCs9YbEBCtJKCJhnCKNeNfU/sWq6bvWKuYXiDnVoby8emSrGKIO1hPz+jXnc5Jf

In [101]:
if res.tool_calls:
    tool_messages = []
    for tool_call in res.tool_calls:
        # Assuming you have a dictionary mapping tool names to actual functions
        tool_name = tool_call.get("name")
        tool_args = tool_call.get("args")


In [102]:
# tool_msg = ToolMessage(
#     content='below is the image',
#     tool_call_id=tool_call.get("id")
# )
# input_list.append(tool_msg)

tool_msg = ToolMessage(
    content=[
        {
            "type": "input_image",
            "image_url": image_url
        },
    ],
    tool_call_id=tool_call.get("id")
)
input_list.append(tool_msg)
input_list

[{'role': 'user',
  'content': 'Get the poster of a random things from internet and explain what you see in the image in detail and tell me no of peoples you see in the image. if you are not able to see the image clearly tell so'},
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_random_image', 'arguments': '{"topic": "random things"}'}, '__gemini_function_call_thought_signatures__': {'315758b7-4a16-4276-b0bb-3b5e474c747f': 'CvgKAXLI2ny7ritsvkh+nF2qfNTFuf7VyzvZrwMgiUBSHvhXBOYr3Xvb5sG9I4d4T1bsz0ZN8YHlxH2q6YXxKktQF72yszMCmwvPmOMAcESyQU/IyDHp3fZ8T9hlOPPhsgBXa4N6XtBCDrwlLMEOXqMPWmjRTM6PPjMCTnK/Ts2qncv5Py78s7XtNtKQ7KkImFbX9ADQ7tFzi4xgoCxlUdXK3OBD+ge4zh8nqX+6ZSYhmC9exOlHtPKKILJM2r+z2qI86+4K+UCoYlubR6iazZQj36KO/wEuKjnuGuTe3zI1ySXjBKW68MB0nYwCEFHQytALHKWEh+9TWcy9jB6SWZVuFTPh0T5DRKt/ibo6MImY8EIh9DotAj8yDTdFgehNWfFMs7sD2PFl8jGEQpGIyWBzKpf3H+PpEVMKHYXM/CNYvH6lnCOZFLcdwCstjRVuTgvEjtzyNmesklNbgt97pxNuvVwAYseuqI84QCs9YbEBCtJKCJhnCKNeNfU/sWq6bvWKuYXiDnVoby8emSrGKIO1hPz+jXnc5Jf

In [103]:
llm.invoke(
    input_list
)

AIMessage(content="In the image, I can see a fluffy white dog that appears to be a Samoyed or a similar breed. The dog is wearing a pair of black-rimmed glasses, which are perched on its snout. The dog's mouth is slightly open, giving it a cheerful and friendly expression. The background is a plain, light-colored wall, which makes the dog the central focus of the image.\n\nI can see zero people in this image.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b1e35-9156-7951-83e4-83a9610e7ee6-0', usage_metadata={'input_tokens': 184, 'output_tokens': 92, 'total_tokens': 276, 'input_token_details': {'cache_read': 0}})